# Delphi Demo - CAGs from <span style='color:royalblue; font-style: italic'>Text</span>

*July 30, 2018*

This is a Jupyter notebook created to showcase the design and capabilities of
the Delphi package, available at [https://github.com/ml4ai/delphi](https://github.com/ml4ai/delphi). 

A rendered HTML version of this notebook can also be found at
[`http://vision.cs.arizona.edu/adarsh/export/delphi_demo_wm_pi_meeting_2018.html`](http://vision.cs.arizona.edu/adarsh/export/delphi_demo_wm_pi_meeting_2018.html)

This demo has been tested with the version of Delphi corresponding to the commit hash below.

In [ ]:
!git rev-parse HEAD

## Construct and visualize CAG corresponding to use case

In [ ]:
from delphi.utils import get_data_from_url
import pickle
url = 'http://vision.cs.arizona.edu/adarsh/export/demos/data/pi_mtg_demo.pkl'
sts = pickle.load(get_data_from_url(url)) 

In [ ]:
from delphi.AnalysisGraph import AnalysisGraph
from delphi.visualization import visualize
G = AnalysisGraph.from_statements(sts)
visualize(G, rankdir='LR')

## Inspecting and editing CAGs

In [ ]:
from delphi.manipulation import merge_nodes
merge_nodes(G, 'food_security', 'food_insecurity', same_polarity=False)
visualize(G, rankdir='LR', nodes_to_highlight='food_insecurity')

## Inspecting statements

In [ ]:
import pandas as pd
pd.options.display.max_colwidth=1000
pd.options.display.width=1000
from delphi.inspection import statements
import delphi.jupyter_tools as jt
jt.create_statement_inspection_table(statements(G))

## Removing incorrect edges

In [ ]:
G.remove_edges_from([('food_insecurity', 'drought'), ('food_insecurity', 'food_insecurity')])
visualize(G, rankdir='LR', nodes_to_highlight='food_insecurity')

## Mapping concepts to indicators

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from delphi.quantification import map_concepts_to_indicators
url = 'http://vision.cs.arizona.edu/adarsh/export/demos/data/concept_to_indicator_mapping.txt'
mapping = get_data_from_url(url)
map_concepts_to_indicators(G, 2, mapping)
visualize(G, indicators=True)

In [ ]:
from delphi.random_variables import Indicator
G.nodes['flooding']['indicators'] = [Indicator('PRECIPITATION', 'CYCLES')]
G.nodes['conflict']['indicators'] = G.nodes['conflict']['indicators'][1:]
G.nodes['market']['indicators'] = G.nodes['market']['indicators'][1:]
G.nodes['drought']['indicators'] = None
G.nodes['human_migration']['indicators'] = None
visualize(G, indicators=True)

In [ ]:
from datetime import datetime
from delphi.parameterization import parameterize
date = datetime(2014, 1, 1)
url = 'http://vision.cs.arizona.edu/adarsh/export/demos/data/south_sudan_data.csv'
parameterize(G, datetime(2014, 1,1), get_data_from_url(url))
visualize(G, indicators=True, indicator_values = True,
          graph_label=f'Causal Analysis Graph for South Sudan, {date.year}')

## Infer transition model

In [ ]:
url = 'http://vision.cs.arizona.edu/adarsh/export/demos/data/adjectiveData.csv'
G.infer_transition_model(get_data_from_url(url), 10000)

## Set initial parameters

In [ ]:
from delphi.export import export_default_initial_values
export_default_initial_values(G, variables_file='variables.csv')
s0 = pd.read_csv('variables.csv', index_col=0, header=None,
                 error_bad_lines=False)[1]
s0.loc['∂(flooding)/∂t'] = 0.1
s0.to_csv('variables.csv')
s0

## Execute model

In [ ]:
%matplotlib inline
from delphi.bmi import *
from matplotlib import pyplot as plt
import seaborn as sns
from delphi.utils import _insert_line_breaks

initialize(G, 'variables.csv')
def plot_and_update(G, concept1, concept2, t):
    fig, ax = plt.subplots(1, 4, figsize=(28, 6))
    ax[0].set_title(f't = {t}', fontsize=24)
    ax[0].set_xlim(0.5, 1.5)
    for a in ax: a.tick_params(labelsize=24)
    ax[0].set_xlabel(concept1, fontsize=24)
    ax[1].set_xlabel(concept2, fontsize=24)
    indicator1 = G.nodes[concept1]['indicators'][0]
    indicator2 = G.nodes[concept2]['indicators'][0]
    ax[2].set_xlabel(f"{indicator1.name} ({indicator1.unit})", fontsize=24)
    ax[3].set_xlabel(_insert_line_breaks(f"{indicator2.name} ({indicator2.unit})", 30), fontsize=24)
    
    ax[1].set_xlim(0, 4)
    ax[2].set_xlim(2, 6)
    ax[3].set_xlim(0, 100)
    kde_t=False
    if t > 5: kde_t=True
    
    d1 = [
        d for d in G.nodes[concept1]['rv'].dataset 
        if 0. < d < 100
    ] 
    d2 = [
        d for d in G.nodes[concept2]['rv'].dataset 
        if 0. < d < 4
    ] 
    indicator1.dataset = [
        d for d in G.nodes[concept1]['indicators'][0].dataset 
        if 2 < d < 8
    ] 
    indicator2.dataset = [
        d for d in G.nodes[concept2]['indicators'][0].dataset 
        if 0 < d < 100
    ] 
    sns.distplot(d1, ax=ax[0], kde=kde_t)
    sns.distplot(d2, ax=ax[1], kde=kde_t)
    sns.distplot(indicator1.dataset, ax=ax[2], kde=kde_t)
    sns.distplot(indicator2.dataset, ax=ax[3], kde=kde_t)
    
    plt.tight_layout()
    update(G)
    
for t in range(5):
    plot_and_update(G, 'flooding', 'food_insecurity', t)

In [ ]:
from delphi.inspection import inspect_edge
inspect_edge(G, 'flooding', 'food_insecurity')

## Exploring the unknown unknowns

In [ ]:
url = 'http://vision.cs.arizona.edu/adarsh/export/demos/data/pi_mtg_demo_unfiltered.pkl'
with urllib.request.urlopen(url) as f:
    sts = pickle.load(f) 

In [ ]:
from delphi.subgraphs import *
G = AnalysisGraph.from_statements(sts)
merge_nodes(G, 'food_security', 'food_insecurity', same_polarity=False)
G = get_subgraph_for_concept(G, 'food_insecurity', depth_limit=1)
visualize(G, nodes_to_highlight='food_insecurity')

In [ ]:
G = AnalysisGraph.from_statements(sts)
merge_nodes(G, 'food_security', 'food_insecurity', same_polarity=False)
G = get_subgraph_for_concept_pair(G, 'drought', 'food_insecurity', cutoff=2)
visualize(G, nodes_to_highlight=['drought','food_insecurity'])

## Causal analysis graphs from software

## Original Fortran program

In [ ]:
jt.display(f'../data/program_analysis/crop_yield.f')

In [ ]:
%cd ../delphi/program_analysis/autoTranslate/
!./autoTranslate ../../../data/program_analysis/crop_yield.f
%cd ../../../notebooks/

## AST in XML Format

In [ ]:
jt.display('../delphi/program_analysis/autoTranslate/crop_yield.xml')

## Equivalent Python code

In [ ]:
jt.display('../delphi/program_analysis/autoTranslate/crop_yield.py')

## Extracted lambda functions

In [ ]:
jt.display('../delphi/program_analysis/autoTranslate/lambdas.py')

## DBN-JSON file

In [ ]:
dbn_json = '../delphi/program_analysis/autoTranslate/pgm.json'
jt.display(dbn_json)

## Executable DBN - Loop plate representation

In [ ]:
from delphi.program_analysis.scopes import *
root = Scope.from_json(dbn_json)
A = root.to_agraph()
jt.display_image(A.draw(format='png', prog='dot'))

## High-level representation of CAG from program

In [ ]:
import sys
sys.path.append('../delphi/program_analysis/autoTranslate')
import lambdas
from delphi.program_analysis.ProgramAnalysisGraph import ProgramAnalysisGraph
from delphi.bmi import *
G = ProgramAnalysisGraph.from_agraph(A, lambdas)
initialize(G)
G.visualize(show_values = True)

In [ ]:
update(G)
G.visualize(show_values = True)

In [ ]:
update(G)
G.visualize(show_values = True)

## Sensitivity Analysis

In [ ]:
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import numpy as np
from matplotlib import pyplot as plt
from delphi.utils import compose, rcompose
from delphi.program_analysis.ProgramAnalysisGraph import ProgramAnalysisGraph

def make_plots(n_samples, deterministic = True):
    variables = ('RAIN', 'TOTAL_RAIN', 'YIELD_EST')
    vals = {k:[] for k in variables}
    days = {k:[] for k in variables}
    palette = sns.color_palette()
    colors = {k:palette[i] for i, k in enumerate(vals)}
    fig, axes = plt.subplots(1,len(vals), figsize=(18, 5))
    ax = {k:axes[i] for i, k in enumerate(vals)}

    for _ in range(n_samples):
        G = ProgramAnalysisGraph.from_agraph(A, lambdas)
        if not deterministic:
            G.nodes['MAX_RAIN']['init_fn'] = lambda: np.random.normal(4, 1)
        initialize(G)
        for i in range(1,31):
            update(G)
            for k in vals:
                vals[k].append(G.nodes[k]['value'])
                days[k].append(G.nodes['DAY']['value']-2)

    for k in vals:
        sns.lineplot(days[k], vals[k], ax = ax[k], label=k, color=colors[k])
        ax[k].set_xlabel('DAY', fontsize=20)
        ax[k].set_ylabel(k, fontsize=20)

    plt.tight_layout()

make_plots(10, deterministic=False)